In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import re

# Read the CSV file to examine the structure
df = pd.read_csv('All_States_GE.csv')

print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nData Types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head(3))
print("\nMissing values summary:")
print(df.isnull().sum())

/tmp/ipython-input-1373915269.py:8: DtypeWarning: Columns (17,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('All_States_GE.csv')


Dataset Shape: (91669, 45)

Column Names:
['State_Name', 'Assembly_No', 'Constituency_No', 'Year', 'month', 'Poll_No', 'DelimID', 'Position', 'Candidate', 'Sex', 'Party', 'Votes', 'Candidate_Type', 'Valid_Votes', 'Electors', 'Constituency_Name', 'Constituency_Type', 'Sub_Region', 'N_Cand', 'Turnout_Percentage', 'Vote_Share_Percentage', 'Deposit_Lost', 'Margin', 'Margin_Percentage', 'ENOP', 'pid', 'Party_Type_TCPD', 'Party_ID', 'last_poll', 'Contested', 'Last_Party', 'Last_Party_ID', 'Last_Constituency_Name', 'Same_Constituency', 'Same_Party', 'No_Terms', 'Turncoat', 'Incumbent', 'Recontest', 'MyNeta_education', 'TCPD_Prof_Main', 'TCPD_Prof_Main_Desc', 'TCPD_Prof_Second', 'TCPD_Prof_Second_Desc', 'Election_Type']

Data Types:
State_Name                 object
Assembly_No                 int64
Constituency_No             int64
Year                        int64
month                     float64
Poll_No                     int64
DelimID                     int64
Position                   

**Complete Data Preparation Code**

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import re
from datetime import datetime

def clean_and_prepare_data():
    """
    Complete data preparation pipeline for election data
    """
    print("Starting data preparation...")

    # Read the CSV with proper dtype handling
    df = pd.read_csv('All_States_GE.csv', low_memory=False)

    print(f"Original dataset shape: {df.shape}")

    # 1. DATA CLEANING

    # Handle mixed types by converting to appropriate types
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')
    df['Electors'] = pd.to_numeric(df['Electors'], errors='coerce')
    df['Party_ID'] = pd.to_numeric(df['Party_ID'], errors='coerce')
    df['Last_Party_ID'] = pd.to_numeric(df['Last_Party_ID'], errors='coerce')
    df['No_Terms'] = pd.to_numeric(df['No_Terms'], errors='coerce')

    # Convert boolean columns properly
    bool_columns = ['last_poll', 'Same_Constituency', 'Same_Party', 'Turncoat', 'Incumbent', 'Recontest']
    for col in bool_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.lower().replace({
                'true': True, 'false': False, 'nan': False
            }).fillna(False).astype(bool)

    # 2. HANDLE MISSING VALUES

    # Numerical columns - fill with 0 or median
    numerical_cols = ['month', 'Votes', 'Electors', 'Party_ID', 'Last_Party_ID',
                        'No_Terms', 'Margin', 'Margin_Percentage', 'ENOP']
    for col in numerical_cols:
        if col in df.columns:
            if col in ['Votes', 'Margin']:
                df[col] = df[col].fillna(0)
            else:
                df[col] = df[col].fillna(df[col].median())

    # Categorical columns - fill with appropriate values
    df['Sex'] = df['Sex'].fillna('Unknown')
    df['Party'] = df['Party'].fillna('IND')  # Assume independent if missing
    df['Candidate_Type'] = df['Candidate_Type'].fillna('GEN')
    df['Deposit_Lost'] = df['Deposit_Lost'].fillna('no')
    df['Party_Type_TCPD'] = df['Party_Type_TCPD'].fillna('Unknown')

    # Text columns - fill with 'Not Available'
    text_cols = ['Sub_Region', 'Last_Party', 'Last_Constituency_Name',
                    'MyNeta_education', 'TCPD_Prof_Main', 'TCPD_Prof_Main_Desc',
                        'TCPD_Prof_Second', 'TCPD_Prof_Second_Desc']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].fillna('Not Available')

    # 3. DATA VALIDATION AND CONSISTENCY CHECKS

    # Ensure vote percentages are between 0-100
    percentage_cols = ['Turnout_Percentage', 'Vote_Share_Percentage', 'Margin_Percentage']
    for col in percentage_cols:
        if col in df.columns:
            df[col] = df[col].clip(0, 100)

    # Clean candidate names
    df['Candidate'] = df['Candidate'].str.strip().str.title()

    # Clean party names
    df['Party'] = df['Party'].str.strip()

    # Ensure valid years
    df['Year'] = df['Year'].clip(1950, 2024)

    # 4. CREATE DERIVED COLUMNS FOR ANALYSIS

    # Create winner flag
    df['Is_Winner'] = df['Position'] == 1

    # Create margin categories
    conditions = [
        df['Margin_Percentage'] < 5,
        (df['Margin_Percentage'] >= 5) & (df['Margin_Percentage'] < 10),
        df['Margin_Percentage'] >= 10
    ]
    choices = ['Close Contest', 'Moderate Margin', 'Large Margin']
    df['Margin_Category'] = np.select(conditions, choices, default='Unknown')

    # Create experience categories
    df['Experience_Level'] = pd.cut(df['No_Terms'],
                                    bins=[-1, 0, 2, 5, 100],
                                    labels=['New', 'Experienced', 'Veteran', 'Senior'])

    # Party size categories based on votes
    df['Party_Size'] = pd.cut(df['Votes'],
                                bins=[0, 10000, 50000, 200000, float('inf')],
                                labels=['Small', 'Medium', 'Large', 'Very Large'])

    print(f"Cleaned dataset shape: {df.shape}")
    print("Missing values after cleaning:")
    print(df.isnull().sum().sum())  # Total missing values

    return df

def create_database_schema(df):
    """
    Create SQLite database with optimized schema
    """
    print("\nCreating database schema...")

    # Connect to SQLite database (creates if doesn't exist)
    conn = sqlite3.connect('election_data.db')

    # 1. MAIN ELECTION RESULTS TABLE
    df.to_sql('election_results', conn, if_exists='replace', index=False)

    # 2. CREATE DIMENSION TABLES FOR BETTER NORMALIZATION

    # Party dimension table
    party_dim = df[['Party', 'Party_Type_TCPD', 'Party_ID']].drop_duplicates().reset_index(drop=True)
    party_dim['party_dim_id'] = party_dim.index + 1
    party_dim.to_sql('party_dimension', conn, if_exists='replace', index=False)

    # Candidate dimension table
    candidate_dim = df[['Candidate', 'Sex', 'MyNeta_education',
                        'TCPD_Prof_Main', 'TCPD_Prof_Main_Desc',
                        'TCPD_Prof_Second', 'TCPD_Prof_Second_Desc']].drop_duplicates().reset_index(drop=True)
    candidate_dim['candidate_dim_id'] = candidate_dim.index + 1
    candidate_dim.to_sql('candidate_dimension', conn, if_exists='replace', index=False)

    # Constituency dimension table
    constituency_dim = df[['State_Name', 'Constituency_Name', 'Constituency_Type',
                            'Sub_Region', 'Assembly_No', 'Constituency_No']].drop_duplicates().reset_index(drop=True)
    constituency_dim['constituency_dim_id'] = constituency_dim.index + 1
    constituency_dim.to_sql('constituency_dimension', conn, if_exists='replace', index=False)

    # 3. CREATE AGGREGATION TABLES FOR FAST QUERYING

    # Party performance summary
    party_performance_query = """
        CREATE TABLE IF NOT EXISTS party_performance_summary AS
            SELECT
                    Party,
                    COUNT(*) as total_candidates,
                    SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) as seats_won,
                    AVG(Vote_Share_Percentage) as avg_vote_share,
                    SUM(Votes) as total_votes,
                    SUM(CASE WHEN Deposit_Lost = 'no' THEN 1 ELSE 0 END) as deposits_saved
                FROM election_results
                GROUP BY Party
                ORDER BY seats_won DESC, total_votes DESC
            """
    conn.execute(party_performance_query)

    # State-wise summary
    state_summary_query = """
        CREATE TABLE IF NOT EXISTS state_election_summary AS
            SELECT
                    State_Name,
                    COUNT(DISTINCT Constituency_Name) as total_constituencies,
                    COUNT(*) as total_candidates,
                    AVG(Turnout_Percentage) as avg_turnout,
                    SUM(Valid_Votes) as total_valid_votes,
                    MAX(Electors) as total_electors
                FROM election_results
                GROUP BY State_Name
                ORDER BY total_constituencies DESC
            """
    conn.execute(state_summary_query)

    # Candidate experience analysis
    candidate_experience_query = """
        CREATE TABLE IF NOT EXISTS candidate_experience_analysis AS
            SELECT
                Experience_Level,
                candidate_count,
                winners,
                ROUND(100.0 * winners / candidate_count, 2) as win_rate
            FROM (
                SELECT
                    Experience_Level,
                    COUNT(*) as candidate_count,
                    SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) as winners,
                    AVG(No_Terms) as avg_terms
                FROM election_results
                GROUP BY Experience_Level
            )
            ORDER BY win_rate DESC
        """
    conn.execute(candidate_experience_query)

    # Close contest analysis
    close_contest_query = """
        CREATE TABLE IF NOT EXISTS close_contest_analysis AS
            SELECT
                    State_Name,
                    Constituency_Name,
                    Margin_Category,
                    COUNT(*) as contest_count,
                    AVG(Margin_Percentage) as avg_margin
                FROM election_results
                WHERE Position = 1  -- Only winners
                GROUP BY State_Name, Constituency_Name, Margin_Category
                ORDER BY State_Name, contest_count DESC
            """
    conn.execute(close_contest_query)

    # 4. CREATE INDEXES FOR PERFORMANCE
    indexes = [
        "CREATE INDEX IF NOT EXISTS idx_party ON election_results(Party)",
        "CREATE INDEX IF NOT EXISTS idx_state ON election_results(State_Name)",
        "CREATE INDEX IF NOT EXISTS idx_constituency ON election_results(Constituency_Name)",
        "CREATE INDEX IF NOT EXISTS idx_year ON election_results(Year)",
        "CREATE INDEX IF NOT EXISTS idx_position ON election_results(Position)",
        "CREATE INDEX IF NOT EXISTS idx_winner ON election_results(Is_Winner)"
    ]

    for index_query in indexes:
        conn.execute(index_query)

    # Verify table creation
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print("Created tables:")
    for table in tables:
        print(f" - {table[0]}")

    # Show sample data from main table
    sample_data = pd.read_sql("SELECT * FROM election_results LIMIT 3", conn)
    print("\nSample data from main table:")
    print(sample_data[['State_Name', 'Candidate', 'Party', 'Votes', 'Position']])

    conn.commit()
    conn.close()

    print("\nDatabase creation completed successfully!")

def generate_data_quality_report(df):
    """
    Generate comprehensive data quality report
    """
    print("\n" + "="*50)
    print("DATA QUALITY REPORT")
    print("="*50)

    print(f"Total Records: {len(df):,}")
    print(f"Total Columns: {len(df.columns)}")
    print(f"Total Missing Values: {df.isnull().sum().sum()}")

    # Data completeness
    completeness = (1 - df.isnull().sum() / len(df)) * 100
    print("\nColumn Completeness (%):")
    for col, comp in completeness.items():
        print(f"  {col}: {comp:.1f}%")

    # Unique values
    print(f"\nUnique States: {df['State_Name'].nunique()}")
    print(f"Unique Parties: {df['Party'].nunique()}")
    print(f"Unique Constituencies: {df['Constituency_Name'].nunique()}")

    # Election years
    print(f"\nElection Years: {sorted(df['Year'].unique())}")

    # Winner statistics
    winners = df[df['Position'] == 1]
    print(f"\nTotal Winners: {len(winners):,}")
    print(f"Female Winners: {len(winners[winners['Sex'] == 'F']):,}")
    print(f"Male Winners: {len(winners[winners['Sex'] == 'M']):,}")

def create_query_examples():
    """
    Create example queries for common analysis
    """
    conn = sqlite3.connect('election_data.db')

    queries = {
        "Top 10 parties by seats won": """
            SELECT Party, seats_won, total_votes
            FROM party_performance_summary
            ORDER BY seats_won DESC
            LIMIT 10
            """,

        "States with highest turnout": """
            SELECT State_Name, avg_turnout, total_constituencies
            FROM state_election_summary
            ORDER BY avg_turnout DESC
            LIMIT 10
            """,

        "Close contests by state": """
            SELECT State_Name, Margin_Category, COUNT(*) as contest_count
            FROM close_contest_analysis
            GROUP BY State_Name, Margin_Category
            ORDER BY contest_count DESC
            """,

        "Candidate experience analysis": """
            SELECT Experience_Level, candidate_count, winners,
            ROUND(100.0 * winners / candidate_count, 2) as win_rate
            FROM candidate_experience_analysis
            ORDER BY win_rate DESC
            """
    }

    print("\n" + "="*50)
    print("QUERY EXAMPLES")
    print("="*50)

    for query_name, query in queries.items():
        print(f"\n{query_name}:")
        try:
            result = pd.read_sql(query, conn)
            print(result.head())
        except Exception as e:
            print(f"Error: {e}")

    conn.close()

# MAIN EXECUTION
if __name__ == "__main__":
    # Step 1: Clean and prepare data
    cleaned_df = clean_and_prepare_data()

    # Step 2: Generate data quality report
    generate_data_quality_report(cleaned_df)

    # Step 3: Create database and schema
    create_database_schema(cleaned_df)

    # Step 4: Show query examples
    create_query_examples()

    print("\n" + "="*50)
    print("DATA PREPARATION COMPLETED SUCCESSFULLY!")
    print("="*50)
    print("Database file: election_data.db")
    print("Main table: election_results")
    print("Dimension tables: party_dimension, candidate_dimension, constituency_dimension")
    print("Aggregation tables: party_performance_summary, state_election_summary, etc.")

Starting data preparation...
Original dataset shape: (91669, 45)


/tmp/ipython-input-427313477.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype(str).str.lower().replace({
/tmp/ipython-input-427313477.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype(str).str.lower().replace({
/tmp/ipython-input-427313477.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_op

Cleaned dataset shape: (91669, 49)
Missing values after cleaning:
2274

DATA QUALITY REPORT
Total Records: 91,669
Total Columns: 49
Total Missing Values: 2274

Column Completeness (%):
  State_Name: 100.0%
  Assembly_No: 100.0%
  Constituency_No: 100.0%
  Year: 100.0%
  month: 100.0%
  Poll_No: 100.0%
  DelimID: 100.0%
  Position: 100.0%
  Candidate: 100.0%
  Sex: 100.0%
  Party: 100.0%
  Votes: 100.0%
  Candidate_Type: 100.0%
  Valid_Votes: 100.0%
  Electors: 100.0%
  Constituency_Name: 100.0%
  Constituency_Type: 100.0%
  Sub_Region: 100.0%
  N_Cand: 100.0%
  Turnout_Percentage: 100.0%
  Vote_Share_Percentage: 100.0%
  Deposit_Lost: 100.0%
  Margin: 100.0%
  Margin_Percentage: 100.0%
  ENOP: 100.0%
  pid: 98.8%
  Party_Type_TCPD: 100.0%
  Party_ID: 100.0%
  last_poll: 100.0%
  Contested: 98.8%
  Last_Party: 100.0%
  Last_Party_ID: 100.0%
  Last_Constituency_Name: 100.0%
  Same_Constituency: 100.0%
  Same_Party: 100.0%
  No_Terms: 100.0%
  Turncoat: 100.0%
  Incumbent: 100.0%
  Recont

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import re
from datetime import datetime

def clean_and_prepare_data():
    """
    Complete data preparation pipeline for election data
    """
    print("Starting data preparation...")

    # Read the CSV with proper dtype handling
    df = pd.read_csv('All_States_GE.csv', low_memory=False)

    print(f"Original dataset shape: {df.shape}")

    # 1. DATA CLEANING

    # Handle mixed types by converting to appropriate types
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')
    df['Electors'] = pd.to_numeric(df['Electors'], errors='coerce')
    df['Party_ID'] = pd.to_numeric(df['Party_ID'], errors='coerce')
    df['Last_Party_ID'] = pd.to_numeric(df['Last_Party_ID'], errors='coerce')
    df['No_Terms'] = pd.to_numeric(df['No_Terms'], errors='coerce')

    # Convert boolean columns properly
    bool_columns = ['last_poll', 'Same_Constituency', 'Same_Party', 'Turncoat', 'Incumbent', 'Recontest']
    for col in bool_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.lower().replace({
                'true': True, 'false': False, 'nan': False
            }).fillna(False).astype(bool)

    # 2. HANDLE MISSING VALUES

    # Numerical columns - fill with 0 or median
    numerical_cols = ['month', 'Votes', 'Electors', 'Party_ID', 'Last_Party_ID',
                        'No_Terms', 'Margin', 'Margin_Percentage', 'ENOP']
    for col in numerical_cols:
        if col in df.columns:
            if col in ['Votes', 'Margin']:
                df[col] = df[col].fillna(0)
            else:
                df[col] = df[col].fillna(df[col].median())

    # Categorical columns - fill with appropriate values
    df['Sex'] = df['Sex'].fillna('Unknown')
    df['Party'] = df['Party'].fillna('IND')  # Assume independent if missing
    df['Candidate_Type'] = df['Candidate_Type'].fillna('GEN')
    df['Deposit_Lost'] = df['Deposit_Lost'].fillna('no')
    df['Party_Type_TCPD'] = df['Party_Type_TCPD'].fillna('Unknown')

    # Text columns - fill with 'Not Available'
    text_cols = ['Sub_Region', 'Last_Party', 'Last_Constituency_Name',
                    'MyNeta_education', 'TCPD_Prof_Main', 'TCPD_Prof_Main_Desc',
                        'TCPD_Prof_Second', 'TCPD_Prof_Second_Desc']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].fillna('Not Available')

    # 3. DATA VALIDATION AND CONSISTENCY CHECKS

    # Ensure vote percentages are between 0-100
    percentage_cols = ['Turnout_Percentage', 'Vote_Share_Percentage', 'Margin_Percentage']
    for col in percentage_cols:
        if col in df.columns:
            df[col] = df[col].clip(0, 100)

    # Clean candidate names
    df['Candidate'] = df['Candidate'].str.strip().str.title()

    # Clean party names
    df['Party'] = df['Party'].str.strip()

    # Ensure valid years
    df['Year'] = df['Year'].clip(1950, 2024)

    # 4. CREATE DERIVED COLUMNS FOR ANALYSIS

    # Create winner flag
    df['Is_Winner'] = df['Position'] == 1

    # Create margin categories
    conditions = [
        df['Margin_Percentage'] < 5,
        (df['Margin_Percentage'] >= 5) & (df['Margin_Percentage'] < 10),
        df['Margin_Percentage'] >= 10
    ]
    choices = ['Close Contest', 'Moderate Margin', 'Large Margin']
    df['Margin_Category'] = np.select(conditions, choices, default='Unknown')

    # Create experience categories
    df['Experience_Level'] = pd.cut(df['No_Terms'],
                                    bins=[-1, 0, 2, 5, 100],
                                    labels=['New', 'Experienced', 'Veteran', 'Senior'])

    # Party size categories based on votes
    df['Party_Size'] = pd.cut(df['Votes'],
                                bins=[0, 10000, 50000, 200000, float('inf')],
                                labels=['Small', 'Medium', 'Large', 'Very Large'])

    print(f"Cleaned dataset shape: {df.shape}")
    print("Missing values after cleaning:")
    print(df.isnull().sum().sum())  # Total missing values

    return df

def create_database_schema(df):
    """
    Create SQLite database with optimized schema
    """
    print("\nCreating database schema...")

    # Connect to SQLite database (creates if doesn't exist)
    conn = sqlite3.connect('election_data.db')

    # 1. MAIN ELECTION RESULTS TABLE
    df.to_sql('election_results', conn, if_exists='replace', index=False)

    # 2. CREATE DIMENSION TABLES FOR BETTER NORMALIZATION

    # Party dimension table
    party_dim = df[['Party', 'Party_Type_TCPD', 'Party_ID']].drop_duplicates().reset_index(drop=True)
    party_dim['party_dim_id'] = party_dim.index + 1
    party_dim.to_sql('party_dimension', conn, if_exists='replace', index=False)

    # Candidate dimension table
    candidate_dim = df[['Candidate', 'Sex', 'MyNeta_education',
                        'TCPD_Prof_Main', 'TCPD_Prof_Main_Desc',
                        'TCPD_Prof_Second', 'TCPD_Prof_Second_Desc']].drop_duplicates().reset_index(drop=True)
    candidate_dim['candidate_dim_id'] = candidate_dim.index + 1
    candidate_dim.to_sql('candidate_dimension', conn, if_exists='replace', index=False)

    # Constituency dimension table
    constituency_dim = df[['State_Name', 'Constituency_Name', 'Constituency_Type',
                            'Sub_Region', 'Assembly_No', 'Constituency_No']].drop_duplicates().reset_index(drop=True)
    constituency_dim['constituency_dim_id'] = constituency_dim.index + 1
    constituency_dim.to_sql('constituency_dimension', conn, if_exists='replace', index=False)

    # 3. CREATE AGGREGATION TABLES FOR FAST QUERYING

    # Party performance summary
    party_performance_query = """
        CREATE TABLE IF NOT EXISTS party_performance_summary AS
            SELECT
                    Party,
                    COUNT(*) as total_candidates,
                    SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) as seats_won,
                    AVG(Vote_Share_Percentage) as avg_vote_share,
                    SUM(Votes) as total_votes,
                    SUM(CASE WHEN Deposit_Lost = 'no' THEN 1 ELSE 0 END) as deposits_saved
                FROM election_results
                GROUP BY Party
                ORDER BY seats_won DESC, total_votes DESC
            """
    conn.execute(party_performance_query)

    # State-wise summary
    state_summary_query = """
        CREATE TABLE IF NOT EXISTS state_election_summary AS
            SELECT
                    State_Name,
                    COUNT(DISTINCT Constituency_Name) as total_constituencies,
                    COUNT(*) as total_candidates,
                    AVG(Turnout_Percentage) as avg_turnout,
                    SUM(Valid_Votes) as total_valid_votes,
                    MAX(Electors) as total_electors
                FROM election_results
                GROUP BY State_Name
                ORDER BY total_constituencies DESC
            """
    conn.execute(state_summary_query)

    # Candidate experience analysis
    candidate_experience_query = """
        CREATE TABLE IF NOT EXISTS candidate_experience_analysis AS
            SELECT
                Experience_Level,
                candidate_count,
                winners,
                ROUND(100.0 * winners / candidate_count, 2) as win_rate
            FROM (
                SELECT
                    Experience_Level,
                    COUNT(*) as candidate_count,
                    SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) as winners,
                    AVG(No_Terms) as avg_terms
                FROM election_results
                GROUP BY Experience_Level
            )
            ORDER BY win_rate DESC
        """
    conn.execute(candidate_experience_query)

    # Close contest analysis
    close_contest_query = """
        CREATE TABLE IF NOT EXISTS close_contest_analysis AS
            SELECT
                    State_Name,
                    Constituency_Name,
                    Margin_Category,
                    COUNT(*) as contest_count,
                    AVG(Margin_Percentage) as avg_margin
                FROM election_results
                WHERE Position = 1  -- Only winners
                GROUP BY State_Name, Constituency_Name, Margin_Category
                ORDER BY State_Name, contest_count DESC
            """
    conn.execute(close_contest_query)

    # 4. CREATE INDEXES FOR PERFORMANCE
    indexes = [
        "CREATE INDEX IF NOT EXISTS idx_party ON election_results(Party)",
        "CREATE INDEX IF NOT EXISTS idx_state ON election_results(State_Name)",
        "CREATE INDEX IF NOT EXISTS idx_constituency ON election_results(Constituency_Name)",
        "CREATE INDEX IF NOT EXISTS idx_year ON election_results(Year)",
        "CREATE INDEX IF NOT EXISTS idx_position ON election_results(Position)",
        "CREATE INDEX IF NOT EXISTS idx_winner ON election_results(Is_Winner)"
    ]

    for index_query in indexes:
        conn.execute(index_query)

    # Verify table creation
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print("Created tables:")
    for table in tables:
        print(f" - {table[0]}")

    # Show sample data from main table
    sample_data = pd.read_sql("SELECT * FROM election_results LIMIT 3", conn)
    print("\nSample data from main table:")
    print(sample_data[['State_Name', 'Candidate', 'Party', 'Votes', 'Position']])

    conn.commit()
    conn.close()

    print("\nDatabase creation completed successfully!")

def generate_data_quality_report(df):
    """
    Generate comprehensive data quality report
    """
    print("\n" + "="*50)
    print("DATA QUALITY REPORT")
    print("="*50)

    print(f"Total Records: {len(df):,}")
    print(f"Total Columns: {len(df.columns)}")
    print(f"Total Missing Values: {df.isnull().sum().sum()}")

    # Data completeness
    completeness = (1 - df.isnull().sum() / len(df)) * 100
    print("\nColumn Completeness (%):")
    for col, comp in completeness.items():
        print(f"  {col}: {comp:.1f}%")

    # Unique values
    print(f"\nUnique States: {df['State_Name'].nunique()}")
    print(f"Unique Parties: {df['Party'].nunique()}")
    print(f"Unique Constituencies: {df['Constituency_Name'].nunique()}")

    # Election years
    print(f"\nElection Years: {sorted(df['Year'].unique())}")

    # Winner statistics
    winners = df[df['Position'] == 1]
    print(f"\nTotal Winners: {len(winners):,}")
    print(f"Female Winners: {len(winners[winners['Sex'] == 'F']):,}")
    print(f"Male Winners: {len(winners[winners['Sex'] == 'M']):,}")

def create_query_examples():
    """
    Create example queries for common analysis
    """
    conn = sqlite3.connect('election_data.db')

    queries = {
        "Top 10 parties by seats won": """
            SELECT Party, seats_won, total_votes
            FROM party_performance_summary
            ORDER BY seats_won DESC
            LIMIT 10
            """,

        "States with highest turnout": """
            SELECT State_Name, avg_turnout, total_constituencies
            FROM state_election_summary
            ORDER BY avg_turnout DESC
            LIMIT 10
            """,

        "Close contests by state": """
            SELECT State_Name, Margin_Category, COUNT(*) as contest_count
            FROM close_contest_analysis
            GROUP BY State_Name, Margin_Category
            ORDER BY State_Name, contest_count DESC
            """,

        "Candidate experience analysis": """
            SELECT Experience_Level, candidate_count, winners,
            ROUND(100.0 * winners / candidate_count, 2) as win_rate
            FROM candidate_experience_analysis
            ORDER BY win_rate DESC
            """
    }

    print("\n" + "="*50)
    print("QUERY EXAMPLES")
    print("="*50)

    for query_name, query in queries.items():
        print(f"\n{query_name}:")
        try:
            result = pd.read_sql(query, conn)
            print(result.head())
        except Exception as e:
            print(f"Error: {e}")

    conn.close()

# MAIN EXECUTION
if __name__ == "__main__":
    # Step 1: Clean and prepare data
    cleaned_df = clean_and_prepare_data()

    # Step 2: Generate data quality report
    generate_data_quality_report(cleaned_df)

    # Step 3: Create database and schema
    create_database_schema(cleaned_df)

    # Step 4: Show query examples
    create_query_examples()

    print("\n" + "="*50)
    print("DATA PREPARATION COMPLETED SUCCESSFULLY!")
    print("="*50)
    print("Database file: election_data.db")
    print("Main table: election_results")
    print("Dimension tables: party_dimension, candidate_dimension, constituency_dimension")
    print("Aggregation tables: party_performance_summary, state_election_summary, etc.")

Starting data preparation...
Original dataset shape: (91669, 45)


/tmp/ipython-input-3193932709.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype(str).str.lower().replace({
/tmp/ipython-input-3193932709.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype(str).str.lower().replace({
/tmp/ipython-input-3193932709.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set

Cleaned dataset shape: (91669, 49)
Missing values after cleaning:
2274

DATA QUALITY REPORT
Total Records: 91,669
Total Columns: 49
Total Missing Values: 2274

Column Completeness (%):
  State_Name: 100.0%
  Assembly_No: 100.0%
  Constituency_No: 100.0%
  Year: 100.0%
  month: 100.0%
  Poll_No: 100.0%
  DelimID: 100.0%
  Position: 100.0%
  Candidate: 100.0%
  Sex: 100.0%
  Party: 100.0%
  Votes: 100.0%
  Candidate_Type: 100.0%
  Valid_Votes: 100.0%
  Electors: 100.0%
  Constituency_Name: 100.0%
  Constituency_Type: 100.0%
  Sub_Region: 100.0%
  N_Cand: 100.0%
  Turnout_Percentage: 100.0%
  Vote_Share_Percentage: 100.0%
  Deposit_Lost: 100.0%
  Margin: 100.0%
  Margin_Percentage: 100.0%
  ENOP: 100.0%
  pid: 98.8%
  Party_Type_TCPD: 100.0%
  Party_ID: 100.0%
  last_poll: 100.0%
  Contested: 98.8%
  Last_Party: 100.0%
  Last_Party_ID: 100.0%
  Last_Constituency_Name: 100.0%
  Same_Constituency: 100.0%
  Same_Party: 100.0%
  No_Terms: 100.0%
  Turncoat: 100.0%
  Incumbent: 100.0%
  Recont

**Additional Analysis and Export Code**

In [ ]:
# Additional utility functions for data analysis
def export_cleaned_data():
    """
    Export cleaned data to CSV for external use
    """
    df = pd.read_csv('All_States_GE.csv', low_memory=False)

    # Apply the same cleaning logic
    cleaned_df = clean_and_prepare_data()

    # Export to CSV
    cleaned_df.to_csv('cleaned_election_data.csv', index=False)
    print("Cleaned data exported to 'cleaned_election_data.csv'")

def create_advanced_queries():
    """
    Create more advanced analytical queries
    """
    conn = sqlite3.connect('election_data.db')

    advanced_queries = {
        "Party performance by state": """
            SELECT
                e.State_Name,
                e.Party,
                COUNT(*) as candidates,
                SUM(CASE WHEN e.Position = 1 THEN 1 ELSE 0 END) as winners,
                AVG(e.Vote_Share_Percentage) as avg_vote_share
            FROM election_results e
            GROUP BY e.State_Name, e.Party
            HAVING winners > 0
            ORDER BY e.State_Name, winners DESC
            """,

        "Incumbent performance analysis": """
            SELECT
                Incumbent,
                COUNT(*) as candidates,
                SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) as winners,
                ROUND(100.0 * SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) as win_rate
            FROM election_results
            GROUP BY Incumbent
            """,

        "Gender-wise performance": """
            SELECT
                Sex,
                COUNT(*) as candidates,
                SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) as winners,
                AVG(Vote_Share_Percentage) as avg_vote_share,
                ROUND(100.0 * SUM(CASE WHEN Position = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) as win_rate
            FROM election_results
            GROUP BY Sex
            """,

        "Top constituencies by voter turnout": """
            SELECT
                State_Name,
                Constituency_Name,
                MAX(Turnout_Percentage) as max_turnout,
                MAX(Electors) as total_electors,
                MAX(Valid_Votes) as total_votes
            FROM election_results
            GROUP BY State_Name, Constituency_Name
            ORDER BY max_turnout DESC
            LIMIT 15
            """
    }

    print("\n" + "="*50)
    print("ADVANCED ANALYTICAL QUERIES")
    print("="*50)

    for query_name, query in advanced_queries.items():
        print(f"\n{query_name}:")
        try:
            result = pd.read_sql(query, conn)
            print(result.head(8))
        except Exception as e:
            print(f"Error: {e}")

    conn.close()

# Run additional functions if needed
# export_cleaned_data()
# create_advanced_queries()

**Database Connection and Management Code**

In [ ]:
class ElectionDatabaseManager:
    """
    Class to manage election database operations
    """

    def __init__(self, db_path='election_data.db'):
        self.db_path = db_path
        self.conn = None

    def connect(self):
        """Connect to the database"""
        self.conn = sqlite3.connect(self.db_path)
        return self.conn

    def disconnect(self):
        """Disconnect from database"""
        if self.conn:
            self.conn.close()

    def get_table_info(self, table_name):
        """Get schema information for a table"""
        conn = self.connect()
        query = f"PRAGMA table_info({table_name})"
        schema = pd.read_sql(query, conn)
        self.disconnect()
        return schema

    def get_table_stats(self):
        """Get statistics for all tables"""
        conn = self.connect()
        cursor = conn.cursor()

        # Get all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        stats = {}
        for table in tables:
            table_name = table[0]
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            row_count = cursor.fetchone()[0]
            stats[table_name] = row_count

        self.disconnect()
        return stats

    def custom_query(self, query, params=None):
        """Execute custom query"""
        conn = self.connect()
        try:
            if params:
                result = pd.read_sql(query, conn, params=params)
            else:
                result = pd.read_sql(query, conn)
            return result
        except Exception as e:
            print(f"Query error: {e}")
            return None
        finally:
            self.disconnect()

# Usage example:
def demonstrate_database_usage():
    """
    Demonstrate how to use the database manager
    """
    db_manager = ElectionDatabaseManager()

    print("Database Statistics:")
    stats = db_manager.get_table_stats()
    for table, count in stats.items():
        print(f"  {table}: {count:,} rows")

    print("\nMain Table Schema:")
    schema = db_manager.get_table_info('election_results')
    print(schema[['name', 'type']].head(10))

    # Example custom query
    custom_query = """
        SELECT
            State_Name,
            COUNT(*) as total_candidates,
            SUM(CASE WHEN Sex = 'F' THEN 1 ELSE 0 END) as female_candidates,
            ROUND(100.0 * SUM(CASE WHEN Sex = 'F' THEN 1 ELSE 0 END) / COUNT(*), 2) as female_percentage
        FROM election_results
        GROUP BY State_Name
        ORDER BY female_percentage DESC
        LIMIT 10
        """

    print("\nGender Diversity by State:")
    result = db_manager.custom_query(custom_query)
    print(result)

# Uncomment to run demonstration
# demonstrate_database_usage()